In [1]:
# -*- coding: utf-8 -*-

import pickle as pkl
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import pandas as pd
import numpy as np
import math
import os
import numpy.linalg as la
from acell import preprocess_data,load_assist_data
from tgcn import tgcnCell
from gru import GRUCell 

from visualization import plot_result,plot_error
from sklearn.metrics import mean_squared_error,mean_absolute_error
import matplotlib.pyplot as plt
# from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import time

time_start = time.time()

###### Settings ######
flags = tf.app.flags
FLAGS = flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_float('learning_rate', 0.001, 'Initial learning rate.')
flags.DEFINE_integer('training_epoch', 2500, 'Number of epochs to train.')
flags.DEFINE_integer('gru_units', 100, 'hidden units of gru.')
flags.DEFINE_integer('seq_len', 5, 'time length of inputs.')
flags.DEFINE_integer('pre_len', 1, 'time length of prediction.')
flags.DEFINE_float('train_rate', 0.8, 'rate of training set.')
flags.DEFINE_integer('batch_size', 64, 'batch size.')
flags.DEFINE_string('dataset', '3611817550', 'dataset')
flags.DEFINE_string('model_name', 'tgcn', 'ast-gcn/tgcn/gru')
flags.DEFINE_integer('scheme', 14, 'scheme')
                            #记得改dim
flags.DEFINE_string('noise_name', 'None', 'None or Gauss or Possion')
flags.DEFINE_integer('noise_param', 0, 'Parameter for noise')
#flags.DEFINE_integer('dim', 20, 'DIM')

model_name = FLAGS.model_name
noise_name = FLAGS.noise_name
data_name = FLAGS.dataset
train_rate =  FLAGS.train_rate
seq_len = FLAGS.seq_len
output_dim = pre_len = FLAGS.pre_len
batch_size = FLAGS.batch_size
lr = FLAGS.learning_rate
training_epoch = FLAGS.training_epoch
gru_units = FLAGS.gru_units
#dim = FLAGS.dim
scheme = FLAGS.scheme
PG = FLAGS.noise_param

In [ ]:
###### load data ######
if data_name == '3611817550':
    data, adj = load_assist_data('3611817550', model_name)
#if data_name == 'sh':
#    data, adj = load_sh_data('sh')

### Perturbation Analysis
def MaxMinNormalization(x,Max,Min):
    x = (x-Min)/(Max-Min)
    return x

if noise_name == 'Gauss':
    Gauss = np.random.normal(0,PG,size=data.shape)
    noise_Gauss = MaxMinNormalization(Gauss,np.max(Gauss),np.min(Gauss))
    data = data + noise_Gauss
elif noise_name == 'Possion':
    Possion = np.random.poisson(PG,size=data.shape)
    noise_Possion = MaxMinNormalization(Possion,np.max(Possion),np.min(Possion))
    data = data + noise_Possion
else:
    data = data

time_len = data.shape[0]
num_nodes = data.shape[1]
data1 =np.mat(data,dtype=np.float32)


############## normalization ###############
price_frame = data1
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1), copy=True)
data1 = min_max_scaler.fit_transform(price_frame)
#max_value = np.max(data1)
max_value = 1
#data1  = data1/max_value

#data1.columns = data.columns
if model_name == 'ast-gcn':
    if scheme == 1:
        name = 'add direction dim'
    elif scheme == 2:
        name = 'add beach_width dim'
    elif scheme == 3:
        name = 'add Reservoir_water_level dim'
    elif scheme == 4:
        name = 'add phreatic_line dim'
    elif scheme == 5:
        name = 'add precipitation dim'
    elif scheme == 6:
        name = 'add dynamic attribute dim'
    else:
        name = 'add direction + dynamic attributes dim'

if model_name == 'tgcn':
    name = 'tgcn'
if model_name == 'gru':
    name = 'gru'

print('model:', model_name)
print('scheme:', name)
print('noise_name:', noise_name)
print('noise_param:', PG)

trainX, trainY, testX, testY = preprocess_data(data1, time_len, train_rate, seq_len, pre_len, model_name, scheme)

totalbatch = int(trainX.shape[0]/batch_size)
training_data_count = len(trainX)

def TGCN(_X, _weights, _biases):
    ###
    cell_1 = tgcnCell(gru_units, adj, num_nodes=num_nodes)
    cell = tf.nn.rnn_cell.MultiRNNCell([cell_1], state_is_tuple=True)
    _X = tf.unstack(_X, axis=1)
    outputs, states = tf.nn.static_rnn(cell, _X, dtype=tf.float32)
    m = []
    for i in outputs:
        o = tf.reshape(i,shape=[-1,num_nodes,gru_units])
        o = tf.reshape(o,shape=[-1,gru_units])
        m.append(o)
    last_output = m[-1]
    output = tf.matmul(last_output, _weights['out']) + _biases['out']
    output = tf.reshape(output,shape=[-1,num_nodes,pre_len])
    output = tf.transpose(output, perm=[0,2,1])
    output = tf.reshape(output, shape=[-1,num_nodes])
    return output, m, states

def GRU(_X, weights, biases):
    ###
    cell_1 = GRUCell(gru_units,num_nodes=num_nodes)
    cell = tf.nn.rnn_cell.MultiRNNCell([cell_1], state_is_tuple=True)
    _X = tf.unstack(_X, axis=1)
    outputs, states = tf.nn.static_rnn(cell, _X, dtype=tf.float32)
    m = []
    for i in outputs:
        o = tf.reshape(i,shape=[-1,num_nodes,gru_units])
        o = tf.reshape(o,shape=[-1,gru_units])
        m.append(o)
    last_output = m[-1]
    output = tf.matmul(last_output, weights['out']) + biases['out']
    output = tf.reshape(output,shape=[-1,num_nodes,pre_len])
    output = tf.transpose(output, perm=[0,2,1])
    output = tf.reshape(output, shape=[-1,num_nodes])

    return output, m, states

###### placeholders ######
if model_name == 'ast-gcn':
    if scheme == 1:
        inputs = tf.placeholder(tf.float32, shape=[None, seq_len+1, num_nodes])
    elif scheme == 2:
        inputs = tf.placeholder(tf.float32, shape=[None, seq_len*2+pre_len, num_nodes])
    elif scheme == 3:
        inputs = tf.placeholder(tf.float32, shape=[None, seq_len*2+pre_len, num_nodes])
    elif scheme == 4:
        inputs = tf.placeholder(tf.float32, shape=[None, seq_len*2+pre_len, num_nodes])
    elif scheme == 5:
        inputs = tf.placeholder(tf.float32, shape=[None, seq_len*2+pre_len, num_nodes])
    elif scheme == 6:
        inputs = tf.placeholder(tf.float32, shape=[None, seq_len+pre_len, num_nodes])
    else:
        inputs = tf.placeholder(tf.float32, shape=[None, seq_len*5+pre_len*4+1, num_nodes])#公式为 seq_len *（特征矩阵个数+动态因子个数）+ pre_len *（动态因子个数）+ 1

else:
    inputs = tf.placeholder(tf.float32, shape=[None, seq_len, num_nodes])

labels = tf.placeholder(tf.float32, shape=[None, pre_len, num_nodes])

# Graph weights
weights = {
    'out': tf.Variable(tf.random_normal([gru_units, pre_len], mean=1.0), name='weight_o')}
biases = {
    'out': tf.Variable(tf.random_normal([pre_len]),name='bias_o')}

if model_name == 'gru':
    pred,ttts,ttto = GRU(inputs, weights, biases)
if model_name == 'tgcn':
    pred,ttts,ttto = TGCN(inputs, weights, biases)

y_pred = pred

###### optimizer ######
lambda_loss = 0.0015
Lreg = lambda_loss * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables())
label = tf.reshape(labels, [-1,num_nodes])
##loss
print('y_pred_shape:', y_pred.shape)
print('label_shape:', label.shape)
loss = tf.reduce_mean(tf.nn.l2_loss(y_pred-label) + Lreg)
#loss = tf.reduce_mean(tf.square(label-y_pred))
##rmse
error = tf.sqrt(tf.reduce_mean(tf.square(y_pred-label)))
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)

###### Initialize session ######
variables = tf.global_variables()
saver = tf.train.Saver(tf.global_variables())  
#sess = tf.Session()

#改动态获取GPU
# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.1)
gpu_options = tf.GPUOptions(allow_growth = True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
sess.run(tf.global_variables_initializer())
#out = 'out/%s'%(model_name)
out = 'out/%s_%s'%(model_name,noise_name)
path1 = 'MinMAXTGCNadj2%s_%s_%s_lr%r_batch%r_unit%r_seq%r_pre%r_epoch%r_scheme%r_PG%r'%(model_name,name,data_name,lr,batch_size,gru_units,seq_len,pre_len,training_epoch,scheme,PG)
path = os.path.join(out,path1)
if not os.path.exists(path):
    os.makedirs(path)

###### evaluation ######
def evaluation1(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae = mean_absolute_error(a, b)
    return rmse, mae
def evaluation2(a,b):
    F_norm = la.norm(a-b,'fro')/la.norm(a,'fro')
    r2 = 1-((a-b)**2).sum()/((a-a.mean())**2).sum()
    var = 1-(np.var(a-b))/np.var(a)
    return 1-F_norm, r2, var

x_axe,batch_loss,batch_rmse,batch_pred = [], [], [], []
test_loss,test_rmse,test_mae,test_acc,test_r2,test_var,test_pred = [],[],[],[],[],[],[]
test_renor_pred = []

for epoch in range(training_epoch):
    for m in range(totalbatch):
        mini_batch = trainX[m * batch_size : (m+1) * batch_size]
        mini_label = trainY[m * batch_size : (m+1) * batch_size]
        _, loss1, rmse1, train_output = sess.run([optimizer, loss, error, y_pred],
                                                 feed_dict = {inputs:mini_batch, labels:mini_label})
        batch_loss.append(loss1)
        batch_rmse.append(rmse1 * max_value)

     # Test completely at every epoch
    loss2, rmse2, test_output = sess.run([loss, error, y_pred],
                                         feed_dict = {inputs:testX, labels:testY})

    testoutput = np.abs(test_output)
    test_label = np.reshape(testY,[-1,num_nodes])
    
# ############## normalization ###############
#     price_frame = pd.DataFrame(pd.read_excel(r'./data_35-5_under10float/3611817550_feature_matrix_X.xlsx', header=0))
#     min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1), copy=True)
#     x_minmax = min_max_scaler.fit_transform(price_frame)
    
    acc, r2_score, var_score = evaluation2(test_label, testoutput)
    
############## renormalization ###############  
#     test_label1 = test_label * max_value
#     test_output1 = testoutput * max_value
    test_label1 = min_max_scaler.inverse_transform(test_label)
    test_output1 = min_max_scaler.inverse_transform(testoutput)
    
    rmse, mae = evaluation1(test_label1, test_output1)
    test_loss.append(loss2)
#     test_rmse.append(rmse * max_value)
#     test_mae.append(mae * max_value)
    test_rmse.append(rmse)
    test_mae.append(mae)
    test_acc.append(acc)
    test_r2.append(r2_score)
    test_var.append(var_score)
    test_pred.append(test_output1)
    test_renor_pred.append(testoutput)
    
    print('Iter:{}'.format(epoch),
          'train_rmse:{:.4}'.format(batch_rmse[-1]),
          'test_loss:{:.4}'.format(loss2),
          'test_rmse:{:.4}'.format(rmse),
          'test_acc:{:.4}'.format(acc))
    
    if (epoch % 500 == 0):        
        saver.save(sess, path+'/model_100/ASTGCN_pre_%r'%epoch, global_step = epoch)
        
time_end = time.time()
print(time_end-time_start,'s')

############## visualization ###############
#x = [i for i in range(training_epoch)]
b = int(len(batch_rmse)/totalbatch)
batch_rmse1 = [i for i in batch_rmse]
train_rmse = [(sum(batch_rmse1[i*totalbatch:(i+1)*totalbatch])/totalbatch) for i in range(b)]
batch_loss1 = [i for i in batch_loss]
train_loss = [(sum(batch_loss1[i*totalbatch:(i+1)*totalbatch])/totalbatch) for i in range(b)]
#test_rmse = [float(i) for i in test_rmse]

index = test_rmse.index(np.min(test_rmse))
test_result = test_pred[index]
test_renor_pred = test_renor_pred[index]
var = pd.DataFrame(test_result)
var2 = pd.DataFrame(test_label1)
var.to_csv(path+'/test_prediction.csv',index = False,header = False)
var2.to_csv(path+'/test_true.csv',index = False,header = False)
plot_result(test_result,test_label1,path)
plot_error(train_rmse,train_loss,test_rmse,test_acc,test_mae,path)
evalution = []
evalution.append(np.min(test_rmse))
evalution.append(test_mae[index])
evalution.append(test_acc[index])
evalution.append(test_r2[index])
evalution.append(test_var[index])
evalution = pd.DataFrame(evalution)
evalution.to_csv(path+'/evalution_all.csv',index=False,header=None)

print('model_name:', model_name)
print('scheme:', scheme)
print('name:', name)
print('noise_name:', noise_name)
print('PG:', PG)
print('min_rmse:%r'%(np.min(test_rmse)),
      'min_mae:%r'%(test_mae[index]),
      'max_acc:%r'%(test_acc[index]),
      'r2:%r'%(test_r2[index]),
      'var:%r'%test_var[index])

test_rmse.clear()
test_mae.clear()
test_acc.clear()
test_r2.clear()
test_var.clear()
j=1
for i in range(num_nodes):
    a = test_label1[:,i]
    b = test_result[:,i]
    a = np.array(a)
    a = np.reshape(a,[-1, pre_len])
    b = np.array(np.transpose(np.mat(b)))
    b = b.repeat(pre_len ,axis=1)
    rmse, mae = evaluation1(a, b)
    
    a = test_label[:,i]
    b = test_renor_pred[:,i]
    a = np.array(a)
    a = np.reshape(a,[-1, pre_len])
    b = np.array(np.transpose(np.mat(b)))
    b = b.repeat(pre_len ,axis=1)
    acc,r2,var = evaluation2(a, b)
    if r2<0:
        r2=0
    print('第%r列 TGCN_rmse:%r\n'%(j,rmse),
          '第%r列 TGCN_mae:%r\n'%(j,mae),
          '第%r列 TGCN_acc:%r\n'%(j,acc),
          '第%r列 TGCN_r2:%r\n'%(j,r2),
          '第%r列 TGCN_var:%r\n'%(j,var))
    print('\n')
    test_rmse.append(rmse)
    test_mae.append(mae)
    test_acc.append(acc)
    test_r2.append(r2)
    test_var.append(var)
    j += 1
rmse = np.array(test_rmse)
mae = np.array(test_mae)
acc = np.array(test_acc)
r2 = np.array(test_r2)
var = np.array(test_var)
rmse = np.mean(rmse)
mae = np.mean(mae)
acc = np.mean(acc)
r2 = np.mean(r2)
var = np.mean(var)
print('mean_rmse:%r'%(rmse),
      'mean_mae:%r'%(mae),
      'meam_acc:%r'%(acc),
      'mean_r2:%r'%(r2),
      'mean_var:%r'%var)
evalution2 = []
evalution2.append(rmse)
evalution2.append(mae)
evalution2.append(acc)
evalution2.append(r2)
evalution2.append(var)
evalution2 = pd.DataFrame(evalution2)
evalution2.to_csv(path+'/evalution_mean.csv',index=False,header=None)


#记得回溯minmax归一化结构